### 연습1. 셰익스피어의 책을 읽고 셰익스피어를 따라하는 TextGenerator를 만들어 봅시다.

1. 책을 깔끔하게 정리해서 불러옵니다.
2. 필요한 특수문자 선별,소문자화, 공백정리, 시작토큰 추가, 종료토큰 추가를 해줍니다.
<img src="../data/ex7/17_1.png" width="40%" height="40%">
3. 위 과정에서 빈 문장들을 정리합니다.
4. 단어사전을 제작하고 토큰화하여 임베딩합니다.
5. LSTM을 이용하여 단어가 들어가면 단어사전의 개수만큼 multi_class 분류를 통해 다음 단어를 예측합니다.
6. 첫 단어를 주고 이후 단어를 예측, 그 이후.. \<end> 토큰이 나오거나 max_len 에 도달할때까지 다음 단어를 예측합니다.

7. 결과
    1. LSTM layer가 한개인 모델에서 예측 (10 epochs)
|모델|결과물|
|:--:|:---:|
|<img src="../data/ex7/17_2.png" width="100%" height="100%">| <img src="../data/ex7/17_3.png" width="80%" height="80%">|
    2. LSTM layer가 두개인 모델에서 예측 (10 epochs)
|모델|결과물|
|:---:|:---:|
|<img src="../data/ex7/17_4.png" width="80%" height="80%">| <img src="../data/ex7/17_5.png" width="80%" height="80%">| 
    3. LSTM layer가 두개인 모델에서 예측 (20 epochs)
|결과물|
|:---:|
|<img src="../data/ex7/17_6.png" width="80%" height="80%">|

결과를 보면 확실히 2개의 레이어를 쓰는것이 좋아보인다.

또한 10 epochs를 학습했을때는 모르는 단어가 보이는데 20 epochs에서는 모르는 단어를 거의 예측하지 않는것 같다.

이를 과적합으로 봐야할지는 모르겠지만 20 epochs에서 과적합이 일어난 것은 아닌것으로 보인다

### 연습2. 노래 가사를 읽고 노래 가사를 써봅시다.

위 연습1 의 과정을 따라하지만 Train, Valid 데이터를 나눠서 학습하면서 Valid_loss를 모니터링 합니다.

1. 모델은 LSTM layer 2개, Epoch = 20으로 진행합니다.

    |padding위치|결과|
    |:--:|:--:|
    |<img src="../data/ex7/17_7.png" width="80%" height="80%">|<img src="../data/ex7/17_8.png" width="80%" height="80%">|
    |<img src="../data/ex7/17_9.png" width="80%" height="80%">|<img src="../data/ex7/17_10.png" width="80%" height="80%">|
   
    결과를 토대로 보면 pre,padding이 훨씬 더 좋은 결과를 보이는 것같다. 특히 문장의 max_len까지 결과를 뽑아주는 것이 좋아보인다.
    
    웃긴것은 fuck은 랩이랑 연관이 있어보이고, fuck that nigga가 붙어서 예측되는것이다.
    

# 데이터 다듬기

데이터 확인

In [27]:
# os: Operating System의 줄임말로, 운영체제에서 제공되는 여러 기능을 파이썬에서 사용할 수 있도록 함 (Ex. 디렉토리 경로 이동, 시스템 환경 변수 가져오기 등)
# re: Regular Expression의 줄임말로, 파이썬 정규표현식을 사용하기 위한 모듈
# numpy(NumPy): 행렬이나 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 라이브러리. 데이터 구조 외에도 수치 계산을 위해 효율적으로 구현된 기능을 제공
# tensorflow(TensorFlow): 데이터 흐름 프로그래밍을 위한 라이브러리. 데이터 그래프를 수치적으로 연산하기 위해 구글에서 만든 오픈 소스 라이브러리.
import os, re 
import numpy as np
import tensorflow as tf


# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = '../data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# sample
for i in raw_corpus[:20]:
    print(i)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


문장 전처리

In [2]:
# 문장 전처리 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2. 특수문자 양쪽에 공백을 넣고
    sentence = re.sub(r'[" "]+', " ", sentence) # 3. 여러개의 공백은 하나의 공백으로 바꿉니다
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    sentence = sentence.strip() # 5. 다시 양쪽 공백을 지웁니다
    sentence = '<start> ' + sentence + ' <end>' # 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 필요없는 문장제거
    if len(sentence) == 0: continue # 없는 문장
    if sentence[-1] == ":": continue # 끝이 : 인 문장
    
    # 문장 cleansing
    preprocessed_sentence = preprocess_sentence(sentence)
    # 담기
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

 단어 사전 생성 및 문장 토큰화

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

def tokenize(corpus):
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, # 토큰화할 단어 수
        filters=' ', # 문장을 정제할 필터함수
        oov_token="<unk>" # 모르는 단어 처리
    )
    # 사전 제작
    tokenizer.fit_on_texts(corpus)
    # 토큰화된 문장
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 문장의 길이 맞추기
    # padding='pre'은 문장 앞에 padding, 'post'는 문장 뒤
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)  
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

# sample
print("토큰화된 텐서 sample 3개")
print(tensor[:3, :10])

# 단어 사전 sample
print("\n단어 사전 단어 sample 10개")
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

    
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

# input, target - sample
print("\ninput, target 샘플")
print(src_input[0])
print(tgt_input[0])

토큰화된 텐서 sample 3개
[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]

단어 사전 단어 sample 10개
1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of

input, target 샘플
[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0]


# 언어 모델 학습

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# 단어 사전 크기 ( 7000 + 1 <0:pad> )
VOCAB_SIZE = tokenizer.num_words + 1   

# [input x, target y] 형태의 데이터셋 생성
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(256, 19), dtype=tf.int32, name=None), TensorSpec(shape=(256, 19), dtype=tf.int32, name=None))>

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(VOCAB_SIZE, embedding_size , hidden_size)

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
# class api로 모델을 부를때는 데이터를 넣어줘야 summary를 확인할 수 있습니다.
model(src_sample)

model.summary()

Model: "text_generator_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     multiple                  1792256   
                                                                 
 lstm_6 (LSTM)               multiple                  5246976   
                                                                 
 lstm_7 (LSTM)               multiple                  8392704   
                                                                 
 dense_4 (Dense)             multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


 학습

In [22]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, # 기본값은 False이다. True이면 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=10) # 만들어둔 데이터셋으로 모델을 학습한다. 10번 학습을 반복하겠다는 의미다.

Epoch 1/10
93/93 [==============================] - 17s 160ms/step - loss: 2.3458
Epoch 2/10
93/93 [==============================] - 15s 162ms/step - loss: 2.2542
Epoch 3/10
93/93 [==============================] - 15s 164ms/step - loss: 2.1935
Epoch 4/10
93/93 [==============================] - 15s 164ms/step - loss: 2.1289
Epoch 5/10
93/93 [==============================] - 15s 165ms/step - loss: 2.0595
Epoch 6/10
93/93 [==============================] - 15s 166ms/step - loss: 1.9843
Epoch 7/10
93/93 [==============================] - 15s 166ms/step - loss: 1.9047
Epoch 8/10
93/93 [==============================] - 15s 166ms/step - loss: 1.8257
Epoch 9/10
93/93 [==============================] - 16s 167ms/step - loss: 1.7436
Epoch 10/10
93/93 [==============================] - 16s 167ms/step - loss: 1.6614


# 문장 생성

In [14]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        predict = model(test_tensor) # 1. 입력받은 문장의 텐서를 입력합니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        
        # 4.모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [23]:
first_ward_list = ["he","she","you","i"]
for ward in first_ward_list:
    text = generate_text(model, tokenizer, init_sentence=f"<start> {ward}") # 시작문장으로 he를 넣어 문장생성 함수 실행
    print(text)

<start> he is not fourteen . <end> 
<start> she is not fourteen . <end> 
<start> you are welcome , sir . <end> 
<start> i ll tell you what you will , sir , i pray you , <end> 


---

# EX TASK

* 데이터 불러오기

In [1]:
import os
import re
from glob import glob

import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
txt_file_path = '../data/lyrics/*' #os.getenv(x)함수는 환경 변수x의 값을 포함하는 문자열 변수를 반환합니다. txt_file_path 에 "/root/aiffel/lyricist/data/lyrics/*" 저장

txt_list = glob(txt_file_path) #txt_file_path 경로에 있는 모든 파일명을 리스트 형식으로 txt_list 에 할당

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='UTF8') as f:
        try:
            raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
            raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.
        except:
            print("read fail")

In [3]:
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])
print("Examples:\n", raw_corpus[-20:])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation', "Baby, can't you read the signs? I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind', "I ain't Mr. Right But if you're looking for fast love", "If that's love in your eyes", "It's more than enough", 'Had some bad love', "So fast love is all that I've got on my mind Ooh, ooh", 'Ooh, ooh Looking for some affirmation', 'Made my way into the sun', 'My friends got their ladies', "And they're all having babies", "I just wanna have some fun I won't bore you with the details, baby", "I don't even wanna waste your time", "Let's just say that maybe", 'You could help me ease my mind']
Examples:
 ['This is the new America', 'We are the new America', 'This is the new America', 'We are the new America', 'This is the new America', 'We are the new America Breathe out, breathe in', 'American oxy

* 문장 전처리

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip() 
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue 
    if sentence[-1] == ":": continue 

    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

* 토큰화

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 단어 수
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre',maxlen=20) # 최대 길이 15, 앞쪽 padding
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

* TTS

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, train_size=0.8)

In [8]:
enc_train[0], dec_train[0], enc_val[0], dec_val[0]

(array([  0,   0,   2, 330,   4,  50,   7,  91, 297,  13, 675,   4,  50,
          7,  91, 297,  13, 966,  20]),
 array([  0,   2, 330,   4,  50,   7,  91, 297,  13, 675,   4,  50,   7,
         91, 297,  13, 966,  20,   3]),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    2,    5,   76,
         171,  268,   17,   11,   16,    9, 1879,    4]),
 array([   0,    0,    0,    0,    0,    0,    0,    2,    5,   76,  171,
         268,   17,   11,   16,    9, 1879,    4,    3]))

* 언어 모델 학습

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

train_dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
train_dataset

test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
test_dataset = test_dataset.cache()
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out

embedding_size = 4000
hidden_size = 256
model = TextGenerator(VOCAB_SIZE, embedding_size , hidden_size)

for src_sample, tgt_sample in train_dataset.take(1): break

model(src_sample)
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  48004000  
                                                                 
 lstm (LSTM)                 multiple                  4359168   
                                                                 
 lstm_1 (LSTM)               multiple                  525312    
                                                                 
 dense (Dense)               multiple                  3084257   
                                                                 
Total params: 55,972,737
Trainable params: 55,972,737
Non-trainable params: 0
_________________________________________________________________


In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_dataset, validation_data=test_dataset, epochs=20)

Epoch 1/20
549/549 [==============================] - 50s 87ms/step - loss: 3.4309 - val_loss: 2.8994
Epoch 2/20
549/549 [==============================] - 47s 86ms/step - loss: 2.6909 - val_loss: 2.5614
Epoch 3/20
549/549 [==============================] - 47s 86ms/step - loss: 2.4981 - val_loss: 2.4554
Epoch 4/20
549/549 [==============================] - 47s 86ms/step - loss: 2.4093 - val_loss: 2.3947
Epoch 5/20
549/549 [==============================] - 47s 86ms/step - loss: 2.3352 - val_loss: 2.3390
Epoch 6/20
549/549 [==============================] - 47s 86ms/step - loss: 2.2669 - val_loss: 2.2955
Epoch 7/20
549/549 [==============================] - 47s 86ms/step - loss: 2.2117 - val_loss: 2.2618
Epoch 8/20
549/549 [==============================] - 47s 86ms/step - loss: 2.1559 - val_loss: 2.2298
Epoch 9/20
549/549 [==============================] - 47s 86ms/step - loss: 2.1104 - val_loss: 2.2004
Epoch 10/20
549/549 [==============================] - 47s 86ms/step - loss: 2.058

* 문장생성

In [11]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    while True:
        predict = model(test_tensor) # 1. 입력받은 문장의 텐서를 입력합니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1) # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        
        # 4.모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [12]:
first_ward_list = ["he","she","you","i","what","how","love","fuck","why","shut","hey","my","Got"]
for ward in first_ward_list:
    text = generate_text(model, tokenizer, init_sentence=f"<start> {ward}") # 시작문장으로 he를 넣어 문장생성 함수 실행
    print(text)

<start> he s got a big , i m a <unk> , i m a flirt it s alright <end> 
<start> she s a monster and i m a flirt it s like a <unk> , i m a <end> 
<start> you know that i m not gonna be there for you , i m a flirt with you <end> 
<start> i m the one , yeah , yeah , yeah , yeah , yeah , yeah , yeah <end> 
<start> what i m gonna do what you want with me ? i m not gonna be the one <end> 
<start> how i m going to be with you , i ll be there for you , oh <end> 
<start> love , love , love , love , love , love , love , love , love , <end> 
<start> fuck all the money and the world is on my knees and i m a diamond ring <end> 
<start> why are you ready ? i m not gonna be , i m a flirt now no <end> 
<start> shut up and i m a go getter ! that s all i ve been through . <end> 
<start> hey , hey , hey , hey , hey , hey , hey , hey , hey , <end> 
<start> my love , love , love , love , love , love , love , love , love <end> 
<start> got a matching rollie , that s on the back , and the lincoln jeeps ooh oo